Universidad del valle de Guatemala  
Dpto. Ciencias de la computacion  
Inteligencia Artificial  
Alberto Suriano  

Laboratorio 8
Andres Quinto - 18288  
Marlon Hernández - 15177  

[Repositorio_aqui](https://github.com/AndresQuinto5/IA_LAB08.git)

### Tasks 1 - Teoría  

1. Investigar el **algoritmo AC-3** y su relación con el algoritmo de **backtracking search**  
    El **algoritmo AC-3** es un algoritmo de consistencia de arcos utilizado en problemas de satisfacción de restricciones (CSP). Su objetivo es reducir los dominios de las variables eliminando valores que no tienen soporte, es decir, que no pueden formar parte de una solución consistente con las restricciones. Por otro lado, **el algoritmo de backtracking search** es una técnica de búsqueda que intenta construir una solución paso a paso, retrocediendo cuando encuentra que una asignación de valores no lleva a una solución válida. La relación entre ambos es que **AC-3** puede utilizarse antes del **backtracking** para preprocesar el CSP, reduciendo los dominios y, por lo tanto, el número de asignaciones a probar durante el backtracking  

    Un ejemplo de este algoritmo podria ser, tenes una caja de lápices de colores y quieres asegurarte de que puedes dibujar un arcoíris completo. Pero hay una regla: cada color solo puede ir al lado de ciertos colores. **El algoritmo AC-3** es como un amigo que revisa todos los lápices y se asegura de que cada uno tenga un vecino adecuado antes de empezar a dibujar. Así, cuando comiences a colorear, no te detendrás a mitad de camino porque todos los lápices están en el orden correcto para hacer un arcoíris perfecto.

    **referencias:**
    - [Algoritmo AC-3](https://en.wikipedia.org/wiki/AC-3_algorithm)
    - [Backtracking Algorithms](https://www.freecodecamp.org/news/backtracking-algorithms-recursive-search/)

    
2. Defina en sus propias palabras el término “Arc Consistency”  
    **En mis propias palabras, “Arc Consistency”** se refiere a un estado en el que, para cada par de variables en un CSP que comparten una restricción, cada valor de la primera variable tiene al menos un valor correspondiente en la segunda variable que satisface la restricción entre ellas. Esto asegura que no hay valores aislados que hagan imposible encontrar una solución completa al problema.

    Un ejemplo podria un juego de parejas de cartas. Cada carta tiene un número y debes encontrarle una pareja que tenga el mismo número. **“Arc Consistency”** significa que todas las cartas tienen al menos una pareja posible. Si alguna carta no tuviera pareja, no podrías ganar el juego. Entonces, antes de jugar, revisas todas las cartas para asegurarte de que cada una tiene una pareja y así sabes que el juego se puede ganar.

    **referencias:**
    - [Arc Consistency](https://en.wikipedia.org/wiki/Arc_consistency)
    - [Arc Consistency Explained](https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/)

### Task 2 - CSP con Backtracking, Beam y Local Search

In [ ]:
class CSP:
    def __init__(self, variables, domains, constraints):
        self.variables = variables
        self.domains = domains
        self.constraints = constraints
        
    def is_consistent(self, assignment):
        for constraint in self.constraints:
            if not constraint(assignment):
                return False
        return True

# Variables (exámenes/cursos)
variables = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7']

# Dominios (días posibles)
domains = {var: ['Lunes', 'Martes', 'Miércoles'] for var in variables}

# Asignación de estudiantes a cursos y exámenes
student_courses = {
    'Estudiante1': ['Curso1', 'Curso2', 'Curso3'],
    'Estudiante2': ['Curso2', 'Curso1', 'Curso3'],
    'Estudiante3': ['Curso1', 'Curso3', 'Curso2'],
    'Estudiante4': ['Curso1', 'Curso3', 'Curso3']
}

course_exams = {
    'Curso1': ['E1', 'E4'],
    'Curso2': ['E2', 'E6'],
    'Curso3': ['E3', 'E5', 'E7']
}

# Restricciones
def different_day_constraint(assignment):
    assigned_days = [value for value in assignment.values() if value is not None]
    return len(set(assigned_days)) == len(assigned_days)

def one_exam_per_day_constraint(assignment):
    for student, courses in student_courses.items():
        assigned_exams = [assignment[exam] for course in courses for exam in course_exams[course] if exam in assignment]
        if len(set(assigned_exams)) != len(assigned_exams):
            return False
    return True

def same_course_different_day_constraint(assignment):
    for course, exams in course_exams.items():
        assigned_days = [assignment[exam] for exam in exams if exam in assignment]
        if len(set(assigned_days)) != len(assigned_days):
            return False
    return True

constraints = [
    different_day_constraint,
    one_exam_per_day_constraint,
    same_course_different_day_constraint
]

# Crear una instancia del problema CSP
exam_scheduling_csp = CSP(variables, domains, constraints)

# Ejemplo de asignación
assignment_example = {
    'E1': 'Lunes',
    'E2': 'Martes',
    'E3': 'Miércoles',
    'E4': 'Lunes',
    'E5': 'Martes',
    'E6': 'Miércoles',
    'E7': 'Lunes'
}

# Imprimir la asignación de estudiantes a cursos y exámenes
print("Asignación de estudiantes a cursos y exámenes:")
for student, courses in student_courses.items():
    print(f"\n{student}:")
    for course in courses:
        exams = course_exams[course]
        exam_days = [f"{exam}: {assignment_example[exam]}" for exam in exams if exam in assignment_example]
        print(f"  {course}: {', '.join(exam_days)}")

# Verificar si la asignación de ejemplo cumple con las restricciones
is_valid_assignment = exam_scheduling_csp.is_consistent(assignment_example)

# Imprimir la asignación de ejemplo y su validez
print("\nAsignación de ejemplo:")
for exam, day in assignment_example.items():
    print(f"{exam}: {day}")
print(f"\n¿La asignación cumple con las restricciones? {is_valid_assignment}")

#### Backtracking implementation

In [ ]:
import time

def backtracking_search(csp, assignment={}):
    if len(assignment) == len(csp.variables):
        return assignment
    
    unassigned_variable = [v for v in csp.variables if v not in assignment][0]
    
    for value in csp.domains[unassigned_variable]:
        new_assignment = assignment.copy()
        new_assignment[unassigned_variable] = value
        
        if csp.is_consistent(new_assignment):
            result = backtracking_search(csp, new_assignment)
            if result is not None:
                return result
    
    return None

# Ejecutar el algoritmo de backtracking search
start_time = time.time()
solution = backtracking_search(exam_scheduling_csp)
end_time = time.time()

# Imprimir la solución y el tiempo de ejecución
print("\nSolución encontrada por Backtracking Search:")
if solution is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution.items():
        print(f"{exam}: {day}")
    print(f"\nTiempo de ejecución: {end_time - start_time} segundos")

### Beam search implementation

In [ ]:
import heapq
import time

def beam_search(csp, beam_width, assignment={}):
    if len(assignment) == len(csp.variables):
        return assignment
    
    unassigned_variables = [v for v in csp.variables if v not in assignment]
    
    beam = []
    for variable in unassigned_variables:
        for value in csp.domains[variable]:
            new_assignment = assignment.copy()
            new_assignment[variable] = value
            conflicts = sum(1 for constraint in csp.constraints if not constraint(new_assignment))
            heapq.heappush(beam, (conflicts, new_assignment))
    
    beam = heapq.nsmallest(beam_width, beam)
    
    for _, new_assignment in beam:
        result = beam_search(csp, beam_width, new_assignment)
        if result is not None:
            return result
    
    return None

# Ejecutar el algoritmo de beam search
beam_width = 5
start_time = time.time()
solution = beam_search(exam_scheduling_csp, beam_width)
end_time = time.time()

# Imprimir la solución y el tiempo de ejecución
print("\nSolución encontrada por Beam Search:")
if solution is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution.items():
        print(f"{exam}: {day}")
    print(f"\nTiempo de ejecución: {end_time - start_time} segundos")

### Local search implementation

In [ ]:
import random
import time

def local_search(csp, max_iterations, assignment={}):
    if len(assignment) == 0:
        assignment = {variable: random.choice(csp.domains[variable]) for variable in csp.variables}
    
    for _ in range(max_iterations):
        if csp.is_consistent(assignment):
            return assignment
        
        variable = random.choice(list(assignment.keys()))
        value = random.choice(csp.domains[variable])
        assignment[variable] = value
    
    return None

# Ejecutar el algoritmo de local search
max_iterations = 1000
start_time = time.time()
solution = local_search(exam_scheduling_csp, max_iterations)
end_time = time.time()

# Imprimir la solución y el tiempo de ejecución
print("\nSolución encontrada por Local Search:")
if solution is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution.items():
        print(f"{exam}: {day}")
    print(f"\nTiempo de ejecución: {end_time - start_time} segundos")